In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pickle
import pandas as pd
import logging
from pathlib import Path

import gym
import gym_chrome_dino
from gym_chrome_dino.utils.wrappers import make_dino

import random
from random import randint

import os , sys
import time

#from PIL import Image
#import cv2 #opencv
#import io
#from io import BytesIO
#from IPython.display import clear_output
#import json
#import base64


#########################################################################################################################df = pd.DataFrame(columns = ["Data_Time","Iteration", "Average_Score", "High_Score", "Epsilon"])
column_names = ["Mintues","Seconds","Games","iteration","Average_Score", "High_Score", "Epsilon"]
df = pd.DataFrame(columns = column_names)
df.columns = [col.replace(',', '') for col in df.columns]
# Back up the pervious csv file incase ran by accident
if Path("./dino_log.csv").is_file():
    temp = pd.read_csv("./dino_log.csv")
    temp.to_csv("back_up_" + "dino_log.csv")
    print("File exist, back up to back_up_dino_log.csv")

df.to_csv("./dino_log.csv")

# formatter = logging.Formatter(r'"%(asctime)s",%(message)s')
formatter = logging.Formatter(r'%(message)s')
logger = logging.getLogger("dino_rl")
logger.setLevel(logging.INFO)
filehandler_dbg = logging.FileHandler(logger.name + '-debug.log', mode='w')
fh = logging.FileHandler("./dino_log.csv")

fh.setFormatter(formatter)
logger.addHandler(fh)
#########################################################################################################################
#########################################################################################################################
#os.chdir(os.path.dirname(os.path.abspath(sys.argv[0])))
print ("directory=",os.getcwd())
#########################################################################################################################
class DQN(nn.Module):  #DQN

#     def __init__(self, num_actions):
#         super(DQN, self).__init__()

#         self.conv1 = nn.Conv2d(4, 32, 8, 4)
#         self.relu1 = nn.ReLU(inplace=True)
#         self.conv2 = nn.Conv2d(32, 64, 4, 2)
#         self.relu2 = nn.ReLU(inplace=True)
#         self.conv3 = nn.Conv2d(64, 64, 3, 1)
#         self.relu3 = nn.ReLU(inplace=True)
#         self.fc4 = nn.Linear(3136, 512) #fully connected
#         self.relu4 = nn.ReLU(inplace=True)
#         self.fc5 = nn.Linear(512 , num_actions)

#     def forward(self, x):
#         #print("forward_ok")
#         y = self.conv1(x)
#         y = self.relu1(y)
#         y = self.conv2(y)
#         y = self.relu2(y)
#         y = self.conv3(y)
#         y = self.relu3(y)
#         y = y.view(y.size()[0], -1)
#         y = self.fc4(y)
#         y = self.relu4(y)
#         y = self.fc5(y)

#         return y
    def __init__(self, num_actions):
        super(DQN, self).__init__()

        self.number_of_actions = num_actions
#         self.gamma = 0.99
#         self.final_epsilon = 0.0001
#         self.initial_epsilon = 0.1
#         self.number_of_iterations = 5000000
#         self.replay_memory_size = 10000
#         self.minibatch_size = 32

        self.conv1 = nn.Conv2d(4, 32, 8, 4)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc4 = nn.Linear(3136, 512)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512 , self.number_of_actions)

    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.conv3(y)
        y = self.relu3(y)
        y = y.view(y.size()[0], -1)
        y = self.fc4(y)
        y = self.relu4(y)
        y = self.fc5(y)

        return y
#########################################################################################################################
class DQN_BN(nn.Module):  # DQN with batch normalization

    def __init__(self, num_actions):
        super(DQN_BN, self).__init__()

        self.conv1 = nn.Conv2d(4, 32, 8, 4)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc4 = nn.Linear(3136, 512) #fully connected
        self.relu4 = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512 , num_actions)

    def forward(self, x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = self.bn2(y)
        y = self.relu2(y)
        y = self.conv3(y)
        y = self.bn3(y)
        y = self.relu3(y)
        y = y.view(y.size()[0], -1)
        y = self.fc4(y)
        y = self.relu4(y)
        y = self.fc5(y)

        return y
########################################################################################################################
class DQN_duel(nn.Module):  #DQN + dueling

    def __init__(self, num_actions):
        super(DQN_duel, self).__init__()

        self.conv1 = nn.Conv2d(4, 32, 8, 4)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc4_a = nn.Linear(3136, 512)
        self.relu4_a = nn.ReLU(inplace=True)
        self.fc4_v = nn.Linear(3136, 512)
        self.relu4_v = nn.ReLU(inplace=True)
        self.fc5_a = nn.Linear(512, num_actions)
        self.fc5_v = nn.Linear(512, 1)

    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.conv3(y)
        y = self.relu3(y)
        y = y.view(y.size()[0], -1)
        y_a=self.fc4_a(y)
        y_a=self.relu4_a(y_a)
        y_v=self.fc4_v(y)
        y_v=self.relu4_v(y_v)
        y_a=self.fc5_a(y_a)
        y_v=self.fc5_v(y_v).expand_as(y_a)
        y= y_v + y_a - y_a.mean(1).unsqueeze(1).expand_as(y_a)

        return y
########################################################################################################################
def train(net, target_net, start, device, logger, start_iter=0):
    global alpha, gamma, epsilon_i, batch_size, replay_mem_size, steps, num_actions, epoch_size, update_size
    start_time = time.time()
    hi_score = 0
    games=0
    iteration = start_iter
    replay_memory = []
    epsilon = epsilon_i
    optimizer=optim.RMSprop(net.parameters(), alpha)
    #########################################################
#     env = gym.make('ChromeDino-v0')
    env = gym.make('ChromeDinoNoBrowser-v0') #
    env = make_dino(env, timer=True, frame_stack=True)
    image_0 = env.reset()
    ##################################################################################
    state = convert(image_0).to(device)
    ################## main loop #################################################################
    generation_score = []
    while iteration < steps:
        #print("forward_call")
        output = net(state)[0]
        action = torch.zeros([num_actions], dtype=torch.float32)#.to(device)
        random_action = random.random() <= epsilon
        action_index = [ torch.randint( num_actions, torch.Size([]), dtype=torch.int8 )#.to(device)
                        if random_action
                        else torch.argmax(output)][0]
        action[action_index] = 1
        if action[0] == 1:
            act=0 #do nothing
        elif action[1] == 1:
            act=1 #jump
        ############################################################################
        image_1, reward, terminal, info = env.step(act)
        state_1= convert(image_1).to(device)
        action = action.unsqueeze(0)
        reward = torch.from_numpy(np.array([reward], dtype=np.float32)).unsqueeze(0).to(device)
        ############ REPLAY MEMORY ############################################################
        replay_memory.append((state, action, reward, state_1, terminal))
        if len(replay_memory) > replay_mem_size:
            replay_memory.pop(0)
        batch = random.sample(replay_memory, min( len(replay_memory), batch_size ) )
        ############################################################################################
        state_batch = torch.cat(tuple(t[0] for t in batch)).to(device)
        action_batch = torch.cat(tuple(t[1] for t in batch)).to(device)
        reward_batch = torch.cat(tuple(t[2] for t in batch)).to(device)
        state_1_batch = torch.cat(tuple(t[3] for t in batch)).to(device)
        #print("forward_call")
        output_1_batch = target_net(state_1_batch)#.to(device)
        y_batch = torch.cat(tuple(reward_batch[i] if batch[i][4]
                                  else reward_batch[i] + gamma * torch.max(output_1_batch[i])
                                  for i in range(len(batch))))#.to(device)
        #print("forward_call")
        q_value = torch.sum( net( state_batch ) * action_batch, dim=1 ).to(device)
        optimizer.zero_grad()
        y_batch = y_batch.detach()
        loss = F.smooth_l1_loss(q_value, y_batch)
        loss.backward()
        optimizer.step()
        state = state_1
        ######  print output routine and reset env ###################################################
#         generation_score = []
        score = env.unwrapped.game.get_score()
        #print("terminal=",terminal)
        if terminal == True:
            games +=1
            if games % update_size == 0:
                target_net.load_state_dict(net.state_dict())
                print("games= ",games)
            generation_score.append(score)
            time.sleep(0.1)
            env.reset()
            if score > hi_score:
                hi_score=score
            if games % epoch_size == 0:
                if games == 0:
                    average_score = 0
                    duration = start_time
                else:
                    average_score = sum(generation_score)/len(generation_score)
                    duration = time.time() - start_time
                dino_log(duration, games, iteration,average_score, hi_score, epsilon, logger)
                generation_score = []
#         if games%5 == 0:
#             if games == 0:
#                 average_score = 0
#                 dino_log(games, average_score, hi_score, epsilon, logger)

#             elif len(generation_score) == 0:
#                 pass
#             else:
#                 average_score = sum(generation_score)/len(generation_score)
#                 dino_log(games, average_score, hi_score, epsilon, logger)
# #             generation_score = []
        if iteration % 50000 == 0:
            print("iteration:", iteration, ",score:", score, ",hi_score:", hi_score,
                  ",elapsed time:", time.time() - start, ",epsilon:", epsilon, ",action:",
                  #action_index.cpu().detach().numpy(), ",reward:", reward.numpy()[0][0], ",Q max:",
                  action_index.cpu().detach().numpy(), ",reward:", reward.cpu().numpy()[0][0], ",Q max:",
                  np.max(output.cpu().detach().numpy()))
#             print("iteration:", iteration, ",score:", score, ",hi_score:", hi_score,
#                   ",elapsed time:", time.time() - start, ",epsilon:", epsilon, ",action:",
#                   #action_index.cpu().detach().numpy(), ",reward:", reward.numpy()[0][0], ",Q max:",
#                   action_index.cpu().detach().numpy(), ",reward:", reward.cpu().numpy()[0][0], ",Q max:",
#                   np.max(output.cpu().detach().numpy()), ",avg_score:", average_score)


        if iteration % 50000 == 0:
            torch.save(net, "trained-model/current_model_" + str(iteration) + ".pth")
        if iteration % 50000 == 0:
            print("iteration= ",iteration)
        #############################################################################
        epsilon = linDecay( epsilon )
        iteration += 1

#####################################################################################

def dino_log(duration, games, iteration, average_score, hi_score,epsilon,logger):
#     duration = (int(duration//60)+round((duration%60)/100,2))
    mintue = int(duration//60)
    seconds = int(duration%60)
    space = str(" ")
#     print(str(mintue),seconds)
    logger.info(f"{space},{mintue},{seconds},{games},{iteration},{average_score},{hi_score},{epsilon}")
#####################################################################################
def test(model, device):

    env = gym.make('ChromeDino-v0')
    env = make_dino(env, timer=True, frame_stack=True)
    image_0 = env.reset()
    state = convert(image_0).to(device)
    while True:
        output = model(state)[0]

        action = torch.zeros([model.number_of_actions], dtype=torch.int8)#.to(device)
        action_index = torch.argmax(output)#.to(device)
        action[action_index] = 1

        if action[0] == 1:
            act=0 #do nothing
        elif action[1] == 1:
            act=1 #jump

        image_1, reward, terminal, info = env.step(act)
        state_1= convert(image_1).to(device)

        state = state_1
#####################################################################################
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight, -0.01, 0.01)#.to(device)
        m.bias.data.fill_(0.01)
####################################################################################################
def linDecay(eps):
    global epsilon_i, epsilon_f, epsilon_decay_steps
    start_eps = epsilon_i
    end_eps = epsilon_f
    tot_steps = epsilon_decay_steps
    decay_rate = (start_eps - end_eps) / tot_steps
    if eps > end_eps:
        eps -= decay_rate
    return eps
####################################################################################################
def convert(image):
    state = np.transpose(image, (2, 0, 1))
    state = np.ascontiguousarray(state, dtype=np.float32) / 255
    state = torch.from_numpy(state)
    return state.unsqueeze(0)
#####################################################################################
## MAIN #########

### input parameters ##############
#global alpha, gamma, epsilon_i, epsilon_f, epsilon_decay_steps, batch_size, replay_mem_size, steps, num_actions
alpha = 2e-5
gamma = 0.99
epsilon_i = 0.1
epsilon_f = 0.0001
epsilon_decay_steps =  100000
batch_size = 128
replay_mem_size = 30000
steps = 2500000
epoch_size = 1
num_actions = 2 # 0=do-nothing, 1=jump
update_size = 5
###################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
##comment accordingly if test, train or restart-train mode###
mode="train"
#mode="test"
#mode="restart"

if mode=="train":
    if not os.path.exists('trained-model/'):
        os.mkdir('trained-model/')
    net = DQN( num_actions ).to(device)
    #net = DQN_BN( num_actions ).to(device)
#     net = DQN_duel( num_actions ).to(device)
    target_net = DQN(num_actions).to(device)
    #target_net = DQN_BN(num_actions).to(device)
#     target_net = DQN_duel(num_actions).to(device)
#     net.apply(init_weights)
    target_net.load_state_dict(net.state_dict())
    target_net.eval()

    start = time.time()
    train(net, target_net, start, device, logger).to(device)
elif mode=="restart":
    iterations=250000 #insert the number of iterations of previous run
    net = torch.load('trained-model/restart/restart_model.pth', map_location='cpu').eval()
    net = net.to(device)
    start = time.time()
    train(net, start, device, logger, iterations).to(device)
elif mode=="test":
    net = torch.load('trained-model/trained_agents/current_model_250000.pth', map_location='cpu').eval()
    net = net.to(device)
    test(net, device)#.to(device)
####################################################


File exist, back up to back_up_dino_log.csv
directory= C:\Users\dswon\Desktop
iteration: 0 ,score: 4 ,hi_score: 0 ,elapsed time: 8.707766771316528 ,epsilon: 0.1 ,action: 1 ,reward: 0.0 ,Q max: 0.0027783886
iteration=  0


C:\Users\dswon\anaconda3\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type DQN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


games=  5
games=  10
games=  15
games=  20
games=  25
games=  30
games=  35
games=  40
games=  45
games=  50
games=  55
games=  60
games=  65
games=  70
games=  75
games=  80
games=  85
games=  90
games=  95
games=  100
games=  105
games=  110
games=  115
games=  120
games=  125
games=  130
games=  135
games=  140
games=  145
games=  150
games=  155
games=  160
games=  165
games=  170
games=  175
games=  180
games=  185
games=  190
games=  195
games=  200
games=  205
games=  210
games=  215
games=  220
games=  225
games=  230
games=  235
games=  240
games=  245
games=  250
games=  255
games=  260
games=  265
games=  270
games=  275
games=  280
games=  285
games=  290
games=  295
games=  300
games=  305
games=  310
games=  315
games=  320
games=  325
games=  330
games=  335
games=  340
games=  345
games=  350
games=  355
games=  360
iteration: 50000 ,score: 125 ,hi_score: 495 ,elapsed time: 3474.4812684059143 ,epsilon: 0.05004999999992368 ,action: 0 ,reward: 0.1 ,Q max: 3.329702
iterati